# [OPTIONAL: DATA PARALLELISM](https://pytorch.org/tutorials/beginner/blitz/data_parallel_tutorial.html#sphx-glr-beginner-blitz-data-parallel-tutorial-py)

DataParallel：使用多GPU

```python
device = torch.device("cuda:0")
model.to(device)
mytensor = my_tensor.to(device)
```

 多GPU运行forward、backward

```python
model = nn.DataParallel(model)
```

## 导入、参数

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [5]:
# Parameters and DataLoaders
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


## 随机数据集（Dummy DataSet）

In [10]:
class RandomDataset(Dataset):
    
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)
        
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return self.len
    
rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                         batch_size=batch_size, shuffle=True)

## 简单模型

In [6]:
class Model(nn.Module):
    
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(in_features=input_size, out_features=output_size)
        
    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size {} output size {}"
              .format(input.size(), output.size()))
        return output

## 创建模型及DataParallel

In [8]:
model = Model(input_size, output_size)

if torch.cuda.device_count() > 1:
    print("Let's use {} GPUs!".format(torch.cuda.device_count()))
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)
    
model.to(device)

Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

## 运行模型

In [11]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size {} output_size {}".format(input.size(), output.size()))

	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])
